# Fixed Point

example for what a fixed point class implemented in CoHDL could look like

In [1]:
from __future__ import annotations

from cohdl import (
    Signed,
    Port,
    Entity,
    Temporary,
    BitVector,
    Null,
    static_assert,
    AssignMode
)

from cohdl import std

In [2]:
# helper function to resize signed value with optional zero padding
def resize(signed: Signed, width, zeros=0):
    static_assert(signed.width + zeros <= width)
    if zeros == 0:
        return Temporary[Signed[width]](signed)
    else:
        padded = signed.bitvector @ BitVector[zeros](Null)
        return Temporary[Signed[width]](padded.signed)

class Fixed(std.AssignableType):
    def __init__(self, value: Signed, exponent: int = 0):
        self._val = value.signed
        self._exp = exponent
    
    # define assignment operators (see assignable_type section of the introduction)
    def _assign_(self, source, mode: AssignMode):
        if isinstance(source, Fixed):
            exp_dif = source._exp -self._exp
            static_assert(exp_dif >= 0)
            static_assert(self.max_bit() >= source.max_bit())

            # assign resized and zero padded value according to exponent
            self._val._assign_(resize(source._val, self._val.width, zeros=exp_dif), mode)
        else:
            # if source is a plain Signed
            # wrap it in a fix with zero exponent
            # and reuse above code
            self._assign_(Fixed(source), mode)

    def max_bit(self):
        return self._val.width + self._exp - 1

    # obtain integer part as a signed value
    def integer(self):
        exp = self._exp

        if exp > 0:
            # create integer part by adding zeros
            return resize(self._val, width=exp + self._val.width, zeros=exp)
        else:
            frac = -exp
            if frac >= self._val.width:
                # return zero because there is no integer part
                return Temporary[Signed[1]](0)
            else:
                # create integer part by ignoring fraction bits
                return Temporary(self._val.msb(rest=frac).signed)

    def __add__(self, other: Fixed | Signed):
        if isinstance(other, Fixed):
            if self._exp == other._exp:
                return Fixed(self._val + other._val, self._exp)
            elif self._exp < other._exp:
                # determine required target width
                # from difference in exponents and signal widths
                exp_dif = other._exp - self._exp
                min_bit = self._exp
                max_bit = max(self.max_bit() + exp_dif, other.max_bit())

                result_width = max_bit - min_bit + 1
                return Fixed(
                    resize(self._val, result_width)
                    + resize(other._val, result_width, exp_dif),
                    self._exp,
                )
            else:
                # add in opposite order to reuse above implementation
                return other + self
        else:
            # wrap other in a Fixed with zero exponent
            # to reuse above implementation
            return self + Fixed(other)

    def __str__(self):
        return f"Fixed({self._val}, {self._exp})"

In [3]:
# CoHDL builtins can be used in normal Python code to
# test basic logic (but there is no simulation support
# for synthesizable contexts)

a = Signed[8](5)
b = Signed[8](-10)

print("a =", a)
print("b =", b)

fa = Fixed(a)
fb = Fixed(b, 2)

fr = fa + fb
print(fr.integer(), " == ", 5 + -10*2**2)

a = 5s
b = -10s
Temporary[Signed[9:0]](-35s)  ==  -35


In [4]:
# example usage of Fixed class

class MyEntity(Entity):
    a = Port.input(Signed[8])
    b = Port.input(Signed[8])

    c = Port.output(Signed[16])
    d = Port.output(Signed[16])

    def architecture(self):
        fa = Fixed(self.a, -2)

        @std.concurrent
        def logic():
            fb = Fixed(self.b, 4)
            x = fa + fb
            y = x + self.b
            self.c <<= y.integer()
        
        @std.concurrent
        def logic_assignment():
            fixed_result = Fixed(self.d, -2)

            # Fixed and Signed objects can be assigned
            fixed_result <<= fa
            fixed_result.next = self.a

print(std.VhdlCompiler.to_string(MyEntity))

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity MyEntity is
  port (
    a : in signed(7 downto 0);
    b : in signed(7 downto 0);
    c : out signed(15 downto 0);
    d : out signed(15 downto 0)
    );
end MyEntity;


architecture arch_MyEntity of MyEntity is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_c : signed(15 downto 0);
  signal buffer_d : signed(15 downto 0);
  signal temp : signed(13 downto 0);
  signal temp_1 : std_logic_vector(13 downto 0);
  signal temp_2 : signed(13 downto 0);
  signal value : signed(13 downto 0);
  signal temp_3 : signed(15 downto 0);
  signal temp_4 : std_logic_vector(9 downto 0);
  signal temp_5 : signed(15 downto 0);
  signal value_1 : signed(15 downto 0);
  signal temp_6 : signed(13 downto 0);
  signal temp_7 : signed(15 downto 0);
  signal temp_8 : std_logic_